In [1]:
import sys
import stlearn as st
st.settings.set_figure_params(dpi=300)
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
file = Path("../stimage").resolve()
parent= file.parent
sys.path.append(str(parent))
from PIL import Image
from stimage._utils import gene_plot, Read10X, ReadOldST, tiling, ensembl_to_id
from stimage._model import CNN_NB_multiple_genes, negative_binomial_layer, negative_binomial_loss
from stimage._data_generator import DataGenerator
import tensorflow as tf
import seaborn as sns
sns.set_style("white")
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
# import geopandas as gpd
from sklearn.neighbors import KDTree
from anndata import read_h5ad
from tensorflow.keras import backend as K
import scanpy as sc

import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
from splot.esda import moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation
import pickle
from scipy import stats

def plot_correlation(df, attr_1, attr_2):
    r = stats.pearsonr(df[attr_1], 
                       df[attr_2])[0] **2

    g = sns.lmplot(data=df,
        x=attr_1, y=attr_2,
        height=5, legend=True
    )
    # g.set(ylim=(0, 360), xlim=(0,360))

    g.set_axis_labels(attr_1, attr_2)
    plt.annotate(r'$R^2:{0:.2f}$'.format(r),
                (max(df[attr_1])*0.9, max(df[attr_2])*0.9))
    return g


def calculate_correlation(attr_1, attr_2):
    r = stats.pearsonr(attr_1, 
                       attr_2)[0]
    return r

def calculate_correlation_2(attr_1, attr_2):
    r = stats.spearmanr(attr_1, 
                       attr_2)[0]
    return r

In [2]:
DATA_PATH = Path("/clusterdata/uqxtan9/Xiao/STimage_100run/dataset_breast_cancer_9visium")

adata_all = read_h5ad(DATA_PATH / "all_adata.h5ad")

In [3]:
samples = adata_all.obs["library_id"].unique().tolist()

In [5]:
gene_list=["COX6C","TTLL12", "HSP90AB1", 
           "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

df = pd.DataFrame()


test_sample = ['block1',
 'block2',
 'FFPE']
n_genes = len(gene_list)

In [7]:
adata_all_train_valid = adata_all[adata_all.obs["library_id"].isin(
    adata_all.obs.library_id.cat.remove_categories(test_sample).unique())]

In [12]:
for test_sample in samples:
    test_index = adata_all.obs.library_id == test_sample
    test_dataset_1 = adata_all[test_index,].copy()
    test_gen_1 = tf.data.Dataset.from_generator(
        lambda:DataGenerator(adata=test_dataset_1, 
                      genes=gene_list), 
        output_types=(tf.float32, tuple([tf.float32]*n_genes)), 
        output_shapes=([299,299,3], tuple([1]*n_genes))
    )
    test_gen__1 = test_gen_1.batch(1)
    test_predictions = model.predict(test_gen__1)
    from scipy.stats import nbinom
    y_preds = []
    for i in range(n_genes):
        n = test_predictions[i][:, 0]
        p = test_predictions[i][:, 1]
        y_pred = nbinom.mean(n, p)
        y_preds.append(y_pred)
    test_dataset_1.obsm["predicted_gene"] = np.array(y_preds).transpose()

    test_dataset_1_ = test_dataset_1[:,gene_list].copy()
    test_dataset_1_.X = test_dataset_1_.obsm["predicted_gene"]

    test_dataset_1_.write("./stimage_LOOCV_9visium/pred_{}.h5ad".format(test_sample))

    pred_adata = test_dataset_1_
    test_dataset = test_dataset_1

    for gene in pred_adata.var_names:
        cor_val = calculate_correlation(pred_adata.to_df().loc[:,gene], test_dataset.to_df().loc[:,gene])
        df = df.append(pd.Series([gene, cor_val, test_sample, "STimage"], 
                         index=["Gene", "Pearson correlation", "Slide", "Method"]),
                          ignore_index=True)

    df.to_csv("./stimage_LOOCV_9visium/stimage_cor_{}.csv".format(test_sample))

NameError: name 'model' is not defined

In [3]:
adata_all.var_names

Index(['SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1', 'PERM1', 'HES4', 'ISG15',
       'AGRN', 'RNF223', 'C1orf159',
       ...
       'F8A1', 'FUNDC2', 'CMC4', 'MTCP1', 'VBP1', 'RAB39B', 'CLIC2', 'TMLHE',
       'SPRY3', 'VAMP7'],
      dtype='object', length=14664)

In [4]:
gene_list_st = list(np.load('/clusterdata/uqxtan9/Xiao/STimage/development/stimage_compare_histogene_1000hvg/gene_list.pkl',allow_pickle=True))

In [5]:
len(gene_list_st)

769

In [6]:
gene_list_comm = pd.Index(gene_list_st).intersection(adata_all.var_names)

In [7]:
len(gene_list_comm)

685

In [8]:
with open('./1000hvg_common.pkl', 'wb') as f:
    pickle.dump(gene_list_comm, f)

In [ ]:
gene_list_comm

In [18]:
gene_list_her2 = list(np.load('/clusterdata/uqxtan9/Xiao/STimage/development/Hist2ST/data/her_hvg_cut_1000.npy',allow_pickle=True))


In [19]:
len(gene_list_her2)

785

In [20]:
pd.Index(gene_list_her2).intersection(gene_list_st)

Index(['TNC', 'NUP93', 'PSCA', 'FAM117A', 'SPIN1', 'GAR1', 'PPP1R1A', 'EZH2',
       'COMP', 'TBCA',
       ...
       'PPIH', 'NR4A1', 'MAGEF1', 'SEC14L2', 'HSD17B1', 'MRPS22', 'CXXC5',
       'LTF', 'GNL2', 'NDRG1'],
      dtype='object', length=114)

In [22]:
pd.Index(gene_list_her2).intersection(adata_all.var_names)

Index(['HPS6', 'TNC', 'NR1H2', 'NUP93', 'HNRNPUL2', 'MARS', 'SUGP1', 'DEAF1',
       'WDR13', 'FLNA',
       ...
       'TP53I3', 'TGFBR2', 'UBXN1', 'EIF3F', 'SLC2A8', 'LLGL1', 'SLC16A3',
       'VAV1', 'NDRG1', 'TNFRSF14'],
      dtype='object', length=728)

In [23]:
pd.Index(gene_list_st).intersection(adata_all.var_names).intersection(pd.Index(gene_list_her2).intersection(adata_all.var_names))

Index(['ISG15', 'TCEA3', 'GNL2', 'PPIH', 'HSPB11', 'JAK1', 'IFI44L', 'GBP1',
       'GBP2', 'GBP5',
       ...
       'INTS8', 'NDRG1', 'PSCA', 'CCL19', 'CCL21', 'SPIN1', 'TEX10', 'TNC',
       'NUP214', 'PTGDS'],
      dtype='object', length=101)

In [24]:
marker_gene_list=["COX6C","TTLL12", "HSP90AB1", 
           "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

In [25]:
pd.Index(gene_list_st).intersection(marker_gene_list)

Index(['TFF3', 'CD24'], dtype='object')

In [27]:
pd.Index(gene_list_her2).intersection(marker_gene_list)

Index(['FASN', 'TFF3', 'SPARC', 'CD74'], dtype='object')